# Random Forest
Explore parameter options within the RandomForestClassifier class.  
This notebook is based on RandomForest.16  
Here, the only difference is class_weight = balanced.  

In [1]:
import datetime
print(datetime.datetime.now())
from platform import python_version
print('Python',python_version())
import numpy as np
import pandas as pd
import sklearn   # pip install --upgrade scikit-learn
print('sklearn',sklearn.__version__)
#import tensorflow as tf
#tf.config.list_physical_devices('GPU')

2022-06-12 12:00:46.720095
Python 3.9.6
sklearn 1.1.1


In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.feature_selection import RFE
import joblib # used to dump/load sklearn models
from CellProfiler_Util import CP_Util
from RandomForestUtil import RF_Util

In [3]:
BASE_DIR='/home/jrm/Adjeroh/Naved/CP_80K/'  # Alien
BASE_DIR="D:\\Adjeroh\\Naved\\CP_80K\\"    # Windows
CLASSES=range(0,6)  # use all 6 classes
#CLASSES=[5]  # JUST FOR TESTING
MODELS_DIR='D:\\Adjeroh\\Naved\\CP_80K\\models\\RandomForest.30'

In [4]:
print(datetime.datetime.now())
PREFIX='abc_'
def set_prefix(prefix):
    global PREFIX
    PREFIX=prefix
def get_prefix(col_name):
    global PREFIX
    return PREFIX+col_name  # hard coded for now
def load_all_classes():
    X = None
    y = None
    for i in CLASSES:
        print(datetime.datetime.now())
        print("Process class",i)
        FULL_PATH=BASE_DIR+'Output'+str(i)+'/'
        cp = CP_Util(FULL_PATH)
        cp.train_test_split()
        Xnuc = cp.get_nucleus_rollup()
        set_prefix('Nuc_')
        Xnuc.rename(get_prefix,axis='columns',inplace=True)
        Xrbc = cp.get_RBC_rollup()
        set_prefix('Rbc_')
        Xrbc.rename(get_prefix,axis='columns',inplace=True)
        Xi = Xnuc.join(Xrbc,how='outer')
        size = len(Xi)
        yi = np.ones(size) * i   # e.g. class 3
        if X is None:
            X = Xi
            y = yi
        else:
            X = pd.concat( (X,Xi) )
            y = np.concatenate( (y,yi) )
    X.fillna(0,inplace=True)  
    return X,y
Xtrain,ytrain=load_all_classes()
Xtrain

2022-06-12 12:00:47.992600
2022-06-12 12:00:47.992600
Process class 0
Num WSI in test/train sets: 23 94
Num patches in test/train sets: 8803 36163
2022-06-12 12:01:52.137457
Process class 1
Num WSI in test/train sets: 7 26
Num patches in test/train sets: 2734 10168
2022-06-12 12:02:11.302312
Process class 2
Num WSI in test/train sets: 7 30
Num patches in test/train sets: 2769 11181
2022-06-12 12:02:32.436653
Process class 3
Num WSI in test/train sets: 3 14
Num patches in test/train sets: 1091 5273
2022-06-12 12:02:42.913484
Process class 4
Num WSI in test/train sets: 2 6
Num patches in test/train sets: 813 2384
2022-06-12 12:02:48.264274
Process class 5
Num WSI in test/train sets: 1 3
Num patches in test/train sets: 396 1195


,Nuc_ObjectNumber_count,Nuc_ObjectNumber_mean,Nuc_ObjectNumber_std,Nuc_ObjectNumber_min,Nuc_ObjectNumber_25%,Nuc_ObjectNumber_50%,Nuc_ObjectNumber_75%,Nuc_ObjectNumber_max,Nuc_AreaShape_Area_count,Nuc_AreaShape_Area_mean,...,Rbc_Number_Object_Number_75%,Rbc_Number_Object_Number_max,Rbc_Parent_ShrinkRBC_count,Rbc_Parent_ShrinkRBC_mean,Rbc_Parent_ShrinkRBC_std,Rbc_Parent_ShrinkRBC_min,Rbc_Parent_ShrinkRBC_25%,Rbc_Parent_ShrinkRBC_50%,Rbc_Parent_ShrinkRBC_75%,Rbc_Parent_ShrinkRBC_max
PatchNumber,,,,,,,,,,,,,,,,,,,,,
397,7.0,4.0,2.160247,1.0,2.50,4.0,5.50,7.0,7.0,216.000000,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
398,1.0,1.0,0.000000,1.0,1.00,1.0,1.00,1.0,1.0,200.000000,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
399,1.0,1.0,0.000000,1.0,1.00,1.0,1.00,1.0,1.0,181.000000,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
400,4.0,2.5,1.290994,1.0,1.75,2.5,3.25,4.0,4.0,239.250000,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
401,4.0,2.5,1.290994,1.0,1.75,2.5,3.25,4.0,4.0,237.500000,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1587,7.0,4.0,2.160247,1.0,2.50,4.0,5.50,7.0,7.0,368.285714,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
1588,17.0,9.0,5.049752,1.0,5.00,9.0,13.00,17.0,17.0,519.352941,...,0.00,0.0,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0
1589,19.0,10.0,5.627314,1.0,5.50,10.0,14.50,19.0,19.0,388.315789,...,1.75,2.0,2.0,1.5,0.707107,1.0,1.25,1.5,1.75,2.0


In [5]:
print(datetime.datetime.now())
print("Shuffle...")
Xtrain,ytrain=shuffle(Xtrain,ytrain)  #set random_state for reproducibility

2022-06-12 12:02:53.730596
Shuffle...


In [6]:
print(datetime.datetime.now())
print('Cross-validation...')
model = RandomForestClassifier(class_weight='balanced')
rf1 = RF_Util()
rf1.set_model(model)
rf1.set_train(Xtrain,ytrain)
cv_scores = rf1.cross_validation()
print(cv_scores)
print('mean %.4f +/- %.4f' % (cv_scores.mean(),cv_scores.std()))

2022-06-12 12:02:55.236687
Cross-validation...


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] END .................................................... total time= 8.4min


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  8.4min remaining:    0.0s


[CV] END .................................................... total time= 8.5min
[CV] END .................................................... total time= 8.4min
[CV] END .................................................... total time= 8.4min
[CV] END .................................................... total time= 8.3min
[0.81226864 0.80871799 0.81112118 0.80862798 0.81739196]
mean 0.8116 +/- 0.0032


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed: 42.0min finished


In [7]:
print(datetime.datetime.now())
joblib.dump(rf1.get_model(),MODELS_DIR)

2022-06-12 12:44:53.954851


['D:\\Adjeroh\\Naved\\CP_80K\\models\\RandomForest.30']